# Predicting Solar Panel Adoption - Models Trained with Dataset Variations
#### UC Berkeley MIDS
`Team: Gabriel Hudson, Noah Levy, Laura Williams`

This notebook trains our final model on dataset variations for comparison to our final dataset and model. For more details or explanations about the choices we made here, see our other work in the _Modeling_ folder in this repo.

**Dataset**  
See the _Dataset Variations_ notebook in this _Modeling Folder_ for more information about the datasets used in this notebook.  

This notebook is set up to load the uncompressed dataset stored in a separate _Datasets_ folder that was not uploaded to the repo due to the large size of the uncompressed dataset.

**Model**   
In this notebok we train two Random Forest sequential models:
* Random Forest Classifier to predict presence or absence of residential solar panels  
* Random Forest Regressor to predict residential solar panel density and analyize most important predictive features.

The original raw dataset and idea for the two-stage Random Forest model came from Stanford's DeepSolar team, who recently completed the most accurate detection to date of all solar panels in the US, using satellite imagery and a convolutional neural network:  
http://web.stanford.edu/group/deepsolar/home

In [1]:
# imports
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_predict
import rfpimp as rfp

%matplotlib inline

In [ ]:
# if necessary, install package on your local machine for calculating feature importances
#!pip install rfpimp

In [2]:
# load curated dataset for final model
# deepsolar = pd.read_csv('../Datasets/SolarPrediction_Final_Dataset.csv', index_col=0)

In [2]:
# VARIATION
# load curated dataset with original outcome variable
deepsolar = pd.read_csv('../Datasets/Final_OldOutcomeVar.csv', index_col=0)

In [32]:
# VARIATION
# load curated dataset without latitude or longitude
deepsolar = pd.read_csv('../Datasets/Final_NoLatLon.csv', index_col=0)

In [33]:
print("Dataset rows and dimensions:", deepsolar.shape)

Dataset rows and dimensions: (71305, 58)


## Define outcome and featureset variables

In [34]:
# VARIATION
# set variable for stage 2 Regressor for dataset variation with original outcome variable
# outcome_var_R = 'number_of_solar_system_per_household'

In [35]:
# set variable for stage 2 Regressor for final model
outcome_var_R = 'owner_occupied_solar_system_density'

In [36]:
# create binary outcome variable for stage 1 random forest Classifier
deepsolar['solar_flag']=deepsolar[outcome_var_R].apply(lambda x: int(x>0))

In [37]:
# Confirm values in new outcome variable
print("New binary outcome variable for Stage 1 random forest classifier 'solar_flag' has", 
      deepsolar['solar_flag'].nunique(), "values:", deepsolar["solar_flag"].min(),
      "and", deepsolar["solar_flag"].max())
print("New dataset rows and dimensions:", deepsolar.shape)

New binary outcome variable for Stage 1 random forest classifier 'solar_flag' has 2 values: 0 and 1
New dataset rows and dimensions: (71305, 59)


In [38]:
# set variable for stage 1 Classifier
outcome_var_C = 'solar_flag'

In [39]:
# set variable for feature names
feature_cols = deepsolar.drop(labels=[outcome_var_C, outcome_var_R], axis=1).columns.values
print("Number of features:", len(feature_cols))

Number of features: 57


## Split outcome variables from features

In [40]:
# separate outcome variables and features
X = deepsolar.drop(labels=[outcome_var_C, outcome_var_R], axis=1).values
Y_classifier = deepsolar[outcome_var_C].values
Y_regressor = deepsolar[outcome_var_R].values
print("Full featureset shape is", X.shape)
print("Classifier outcome variable shape:", Y_classifier.shape)
print("Regressor outcome variable shape:", Y_regressor.shape)

Full featureset shape is (71305, 57)
Classifier outcome variable shape: (71305,)
Regressor outcome variable shape: (71305,)


## Train the model using cross-validation

We use 10-fold cross-validation to train our model.  

### Define functions

In [41]:
def create_splits(X, Y_classifier, Y_regressor, n_splits):
    """Create dataset splits for cross-validated hyperparameter tuning.
    
    Input:  dataset features(X), two outcome variables(Y_classifier and Y_regressor), 
            number of splits.
    Output: train and test set as lists of splits
    """
    # set up splits
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=None)
    # initialize variables
    X_trains=[]
    X_tests=[]
    YC_trains=[]
    YC_tests=[]
    YR_trains = []
    YR_tests = []
    # create lists of data splits
    for train_index, test_index in kf.split(X):
        X_trains.append(X[train_index])
        X_tests.append(X[test_index])
        YC_trains.append(Y_classifier[train_index])
        YC_tests.append(Y_classifier[test_index])
        YR_trains.append(Y_regressor[train_index])
        YR_tests.append(Y_regressor[test_index])
    return X_trains, X_tests, YC_trains, YC_tests, YR_trains, YR_tests

In [42]:
def train_2stage_model_CV(n_splits, 
                          outcome_var_C, outcome_var_R, feature_cols,
                          n_C, depth_C, features_C, 
                          n_R, depth_R, features_R,
                          X_trains, X_tests,
                          YC_trains, YC_tests,
                          YR_trains, YR_tests):
    """Two-stage Random Forest model is trained using cross-validation.    
    Stage 1: Classifier predicting residential solar panel presence or absence in a census tract
    Stage 2: Regressor predicting residential solar panel density
    
    Input: hyperparameters for Stage 1 classifier, hyperparameters for Stage 2 regressor,
    dataset splits as created above
    Output: classifer, regressor, combined R squared scores and feature importances for each datasplit
    """
    
    def important_features(model, feature_cols, outcome_var, X_test, Y_test):
        """Calculate important features from each model from within cross-validation model training"""
        importances_permutation = rfp.importances(model, 
                                        pd.DataFrame(X_test,columns=feature_cols), 
                                        pd.DataFrame(Y_test,columns=[outcome_var]))
        importances_standard = model.feature_importances_
        return importances_permutation, importances_standard
    
    # initialize variables
    classifier_scores = []
    regressor_scores = []
    combined_scores = []
    feature_importances_RP = []
    feature_importances_RS = []
    feature_importances_CP = []
    feature_importances_CS = []
    # train model and record output for each model
    for i in range(n_splits):
        # define and fit classifier
        classifier = RandomForestClassifier(n_estimators=n_C, max_depth=depth_C, max_features=features_C, n_jobs=-1)
        classifier.fit(X_trains[i], YC_trains[i])  
        # define and fit regressor
        regressor = RandomForestRegressor(n_estimators=n_R, max_depth=depth_R, max_features=features_R, n_jobs=-1)
        regressor.fit(X_trains[i], YR_trains[i])
        # calculate individual scores
        classifier_score = classifier.score(X_tests[i],YC_tests[i])
        regressor_score = regressor.score(X_tests[i],YR_tests[i])
        # calculate combined model scores
        classifier_preds=classifier.predict(X_tests[i])
        regressor_preds=regressor.predict(X_tests[i])
        final_preds=regressor_preds*classifier_preds
        combined_score = r2_score(YR_tests[i],final_preds)
        # add scores to lists for output
        classifier_scores.append(classifier_score)
        regressor_scores.append(regressor_score)
        combined_scores.append(combined_score)
        # calculate feature importances and add to lists for output
        importances_CP, importances_CS = important_features(classifier, feature_cols, outcome_var_C, 
                                                            X_tests[i], YC_tests[i])
        feature_importances_CP.append(importances_CP)
        feature_importances_CS.append(importances_CS) 
        importances_RP, importances_RS = important_features(regressor, feature_cols, outcome_var_R, 
                                                            X_tests[i], YR_tests[i])
        feature_importances_RP.append(importances_RP)
        feature_importances_RS.append(importances_RS)
       
        print("Finished iteration", i+1)
        
    return classifier_scores, regressor_scores, combined_scores, \
           feature_importances_RP, feature_importances_RS, feature_importances_CP, feature_importances_CS

### Split dataset

In [43]:
# Set number of splits for cross-validation
n_splits = 10
# create splits
X_trains, X_tests, \
YC_trains, YC_tests, \
YR_trains, YR_tests = create_splits(X, Y_classifier, Y_regressor, n_splits)

In [44]:
# Set hyperparameters for classifier
depth_C = 25
features_C = 0.25
n_C = 150
# Set hyperparameters for classifier
depth_R = 25
features_R = 0.35
n_R = 200

In [45]:
# train the model with cross-validation
start = time.time()
classifier_scores, \
regressor_scores, \
combined_scores, \
feature_importances_RP, \
feature_importances_RS, \
feature_importances_CP, \
feature_importances_CS = train_2stage_model_CV(n_splits, 
                                               outcome_var_C, outcome_var_R, feature_cols,
                                               n_C, depth_C, features_C, 
                                               n_R, depth_R, features_R,
                                               X_trains, X_tests,
                                               YC_trains, YC_tests,
                                               YR_trains, YR_tests)
print("Processing time: {} minutes".format(round((time.time() - start)/60, 2)))

Finished iteration 1
Finished iteration 2
Finished iteration 3
Finished iteration 4
Finished iteration 5
Finished iteration 6
Finished iteration 7
Finished iteration 8
Finished iteration 9
Finished iteration 10
Processing time: 22.56 minutes


## Report results

### R squared

This first set of results is from the dataset with the original outcome variable.

In [17]:
print("Average classifier R squared is", np.array(classifier_scores).mean())
print("Average regressor R squared is", np.array(regressor_scores).mean())
print("Average combined R squared is", np.array(combined_scores).mean())

Average classifier R squared is 0.812916342391
Average regressor R squared is 0.714364287294
Average combined R squared is 0.716094604403


This second set of results is from the final dataset (with owner-occupied modified outcome variable) with latitude and longitude removed

In [46]:
print("Average classifier R squared is", np.array(classifier_scores).mean())
print("Average regressor R squared is", np.array(regressor_scores).mean())
print("Average combined R squared is", np.array(combined_scores).mean())

Average classifier R squared is 0.811317543082
Average regressor R squared is 0.72013099663
Average combined R squared is 0.721189813084


All the output regarding important features below is from the final model, not the dataset variations.  Code is retained in this notebook in the event anyone wants to later look at feature importances from the dataset variations.

### Regressor feature importances

Regressor permutation importances

In [17]:
# combine average feature importances from each split in the cross-validation
combined_RP = pd.concat(feature_importances_RP, axis=1, join='inner')
# confirm shape
# print(combined_RP.shape)
# calculate mean and sort
feature_importances_regressor_permutation = combined_RP.mean(axis=1).sort_values(ascending=False)

In [18]:
print("Regressor top 10 feature importances calculated with permutation importance:")
feature_importances_regressor_permutation[:10]

Regressor top 10 feature importances calculated with permutation importance:


Feature
incentive_count_residential        0.174510
median_household_income            0.146967
household_type_family_rate         0.093561
lon                                0.065650
daily_solar_radiation              0.059953
population_density                 0.053259
voting_2016_dem_percentage         0.050848
relative_humidity                  0.046750
electricity_consume_residential    0.043912
lat                                0.026844
dtype: float64

In [19]:
# save permutation importances to file
feature_importances_regressor_permutation.to_csv("FeatureImportances/importances_regressor.csv",
                                                 header=["Importance"])

In [20]:
# if needed, to reload feature importances from CSV
feature_importances_R_from_file = pd.read_csv('FeatureImportances/importances_regressor.csv', index_col=0)

Regressor standard gini importances

In [21]:
# convert list of numpy arrays to pandas dataframes, adding feature names
feature_importances_RS_df = []
for i in range(len(feature_importances_RS)):
    df = pd.DataFrame(feature_importances_RS[i], index=feature_cols, columns=["Importance"])
    feature_importances_RS_df.append(df)
# combine average feature importances from each split in the cross-validation
combined_RS = pd.concat(feature_importances_RS_df, axis=1, join='inner')
# calculate mean and sort
feature_importances_regressor_standard = combined_RS.mean(axis=1).sort_values(ascending=False)

In [22]:
print("Regressor top 10 feature importances calculated with standard gini importance:")
feature_importances_regressor_standard[:10]

Regressor top 10 feature importances calculated with standard gini importance:


median_household_income            0.100206
incentive_count_residential        0.073450
household_type_family_rate         0.071812
population_density                 0.055634
daily_solar_radiation              0.053126
housing_unit_median_gross_rent     0.045929
lon                                0.041750
electricity_consume_residential    0.038059
relative_humidity                  0.035466
health_insurance_none_rate         0.027690
dtype: float64

### Classifier importances

Classifier permutation importances

In [23]:
# combine average feature importances from each split in the cross-validation
combined_CP = pd.concat(feature_importances_CP, axis=1, join='inner')
# calculate mean and sort
feature_importances_classifier_permutation = combined_CP.mean(axis=1).sort_values(ascending=False)

In [24]:
print("Classifier top 10 feature importances calculated with permutation importance:")
feature_importances_classifier_permutation[:10]

Classifier top 10 feature importances calculated with permutation importance:


Feature
population_density             0.04614
heating_fuel_coal_coke_rate    0.00796
lon                            0.00560
race_black_africa_rate         0.00308
sales_tax                      0.00188
education_master_rate          0.00180
daily_solar_radiation          0.00174
relative_humidity              0.00156
travel_time_10_19_rate         0.00156
occupancy_vacant_rate          0.00146
dtype: float64

In [25]:
# save permutation importances to file
feature_importances_classifier_permutation.to_csv("FeatureImportances/importances_classifier.csv",
                                                  header=["Importance"])

In [26]:
# if needed, to reload feature importances from CSV
feature_importances_C_from_file = pd.read_csv('FeatureImportances/importances_classifier.csv', index_col=0)

Classifier standard gini importances

In [27]:
# convert list of numpy arrays to pandas dataframes, adding feature names
feature_importances_CS_df = []
for i in range(len(feature_importances_CS)):
    df = pd.DataFrame(feature_importances_CS[i], index=feature_cols, columns=["Importance"])
    feature_importances_CS_df.append(df)
# combine average feature importances from each split in the cross-validation
combined_CS = pd.concat(feature_importances_CS_df, axis=1, join='inner')
# calculate mean and sort
feature_importances_classifier_standard = combined_CS.mean(axis=1).sort_values(ascending=False)

In [28]:
print("Classifier top 10 feature importances calculated with standard gini importance:")
feature_importances_classifier_standard[:10]

Classifier top 10 feature importances calculated with standard gini importance:


population_density                     0.112130
heating_fuel_coal_coke_rate            0.039757
lon                                    0.036234
housing_unit_median_gross_rent         0.030346
occupancy_vacant_rate                  0.029547
race_black_africa_rate                 0.023555
per_capita_income                      0.023194
education_high_school_graduate_rate    0.023013
education_bachelor_rate                0.022209
relative_humidity                      0.021737
dtype: float64

## Create cross-validated predictions

Note: The predictions are created with models trained on the cross-validated splits created by sklearn's `cross_val_predict` function instead of the cross-validated splits in the function above.  However, the same hyperparameters are used to train the model and we expect the results to be similar without any significant differences.

In [12]:
# define the classifier
classifier_for_preds = RandomForestClassifier(n_estimators=n_C, 
                                              max_depth=depth_C, 
                                              max_features=features_C, 
                                              n_jobs=-1)
# define the regressor
regressor_for_preds = RandomForestRegressor(n_estimators=n_R, 
                                            max_depth=depth_R, 
                                            max_features=features_R, 
                                            n_jobs=-1)

In [13]:
# create cross-validated predictions for the classifier
classifier_preds = cross_val_predict(classifier_for_preds, X, Y_classifier, cv=10)

In [14]:
# create cross-validated predictions for the regressor
regressor_preds = cross_val_predict(regressor_for_preds, X, Y_regressor, cv=10)

In [15]:
# create combined final predictions
final_predictions = classifier_preds*regressor_preds

In [48]:
# connect predictions with census tract FIPS code
final_preds_fips = pd.DataFrame(final_predictions, index=deepsolar.index.values)

In [49]:
final_preds_fips.head()

,0
27145011200,0.000000
27145011301,0.001050
27145011302,0.000811
27145011304,0.000000
27145011400,0.000763


In [51]:
# save to CSV
final_preds_fips.to_csv("Final_Predictions.csv")

In [56]:
# to load final predictions from CSV
final_preds_fips_from_file = pd.read_csv("Final_Predictions.csv", index_col=0)